In [1]:
import pandas as pd
import h3

In [2]:
import sys, os
sys.path.append(os.path.abspath('..'))
%load_ext autoreload
%autoreload 2

from modules.config import *

In [3]:
trips = pd.read_pickle(TRIPS_PATH)
relocations = pd.read_pickle(RELOCATIONS_PATH)
movements = pd.read_pickle(MOVEMENTS_PATH)
trips.head(2)

,p_spot_start,b_battery_pack_start,b_pedelec_battery_start,p_address_start,lat_start,b_lock_types,p_name_start,p_number_start,p_bikes_start,p_uid_start,...,p_booked_bikes_end,lng_end,p_bike_types_end,in_flexzone_end,in_free_flexzone_end,in_charged_flexzone_end,type,min_distance,duration,min_avg_speed
4612,False,NaN,0.0,NaN,51.345996,frame_lock,BIKE 20236,0.0,1,12152153,...,0.0,12.377452,"{""71"": 1}",True,False,True,trip,2.504853,22.0,6.831417
6583,False,NaN,0.0,NaN,51.325287,frame_lock,BIKE 20236,0.0,1,12162631,...,0.0,12.403814,"{""71"": 1}",True,True,True,trip,2.676814,19.0,8.453096


In [4]:
trips["start_hex_id"] = trips.apply(
    lambda row: h3.geo_to_h3(row["lat_start"], row["lng_start"], H3_RESOLUTION), axis=1
)
trips["end_hex_id"] = trips.apply(
	lambda row: h3.geo_to_h3(row["lat_end"], row["lng_end"], H3_RESOLUTION), axis=1
)
relocations["start_hex_id"] = relocations.apply(
	lambda row: h3.geo_to_h3(row["lat_start"], row["lng_start"], H3_RESOLUTION), axis=1
)
relocations["end_hex_id"] = relocations.apply(
	lambda row: h3.geo_to_h3(row["lat_end"], row["lng_end"], H3_RESOLUTION), axis=1
)
movements["start_hex_id"] = movements.apply(
	lambda row: h3.geo_to_h3(row["lat_start"], row["lng_start"], H3_RESOLUTION), axis=1
)
movements["end_hex_id"] = movements.apply(
	lambda row: h3.geo_to_h3(row["lat_end"], row["lng_end"], H3_RESOLUTION), axis=1
)



In [5]:
trips[
    ["start_hex_id", "lat_start", "lng_start", "end_hex_id", "lat_end", "lng_end"]
].head(2)


,start_hex_id,lat_start,lng_start,end_hex_id,lat_end,lng_end
4612,881f1a8cabfffff,51.345996,12.360209,881f1a1643fffff,51.326196,12.377452
6583,881f1a1643fffff,51.325287,12.377082,881f1ab965fffff,51.342644,12.403814


### Aggregate Trips by Hexagons

In [6]:
trips_grouped_spatial = (
    trips.groupby(
        ["start_hex_id", "end_hex_id"]
    )
    .size()
    .to_frame("demand")
)
trips_grouped_spatial.head(2)

,,demand
start_hex_id,end_hex_id,
881f18411dfffff,881f1a8dd3fffff,1
881f184cadfffff,881f1a8d9bfffff,1


In [7]:
trips_grouped_spatial.to_pickle(TRIPS_GROUPED_SPATIAL_PATH)

### Aggregate Trips Spatio-Temporally

In [8]:

trips['datetime_start_floored'] = trips['datetime_start'].dt.floor(f'{TIME_INTERVAL_LENGTH}H')
relocations['datetime_start_floored'] = relocations['datetime_start'].dt.floor(f'{TIME_INTERVAL_LENGTH}H')
movements['datetime_start_floored'] = movements['datetime_start'].dt.floor(f'{TIME_INTERVAL_LENGTH}H')
movements['datetime_end_floored'] = movements['datetime_end'].dt.floor(f'{TIME_INTERVAL_LENGTH}H')

In [9]:
trips.datetime_start_floored.head(2)

4612   2019-01-22 12:00:00
6583   2019-01-23 12:00:00
Name: datetime_start_floored, dtype: datetime64[ns]

In [10]:
trips_grouped = (
    trips.groupby(["datetime_start_floored", "start_hex_id", "end_hex_id"])
    .size()
    .to_frame("demand")
)
relocations_grouped = (
    relocations.groupby(["datetime_start_floored", "start_hex_id", "end_hex_id"])
    .size()
    .to_frame("relocations")
)

# we group movements by both start and end location and time, as we will use this
# to accurately calculate the availability of bicycles at each location at each time
movements_grouped = (
    movements.groupby(
        ["datetime_start_floored", "datetime_end_floored", "start_hex_id", "end_hex_id"]
    )
    .size()
    .to_frame("movements")
)
trips_grouped.head(2)


demand
datetime_start_floored start_hex_id    end_hex_id             
2019-01-20             881f1a1601fffff 881f1a1659fffff       3
                       881f1a1609fffff 881f1a1643fffff       2

In [11]:
trips_grouped.index.names = ["time_interval_start", "start_hex_id", "end_hex_id"]
relocations_grouped.index.names = ["time_interval_start", "start_hex_id", "end_hex_id"]
movements_grouped.index.names = ["time_interval_start", "time_interval_end", "start_hex_id", "end_hex_id"]

In [12]:
trips_grouped.to_pickle(TRIPS_GROUPED_SPATIO_TEMPORAL_PATH)
relocations_grouped.to_pickle(RELOCATIONS_GROUPED_SPATIO_TEMPORAL_PATH)
movements_grouped.to_pickle(MOVEMENTS_GROUPED_SPATIO_TEMPORAL_PATH)